In [1]:
import gc
import os
import shutil
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [88]:
cutoff = 0
Q = 13
TotalSPStates = int((4*cutoff + 2) * Q + 2 * cutoff * (cutoff + 1)) # Total number of single particle states
LargestNumber = 2**TotalSPStates

In [80]:
LargestNumber

1048576

In [89]:
l_max = Q - 0.5 + cutoff
mm = [-l_max + i for i in range(int(2*l_max+1))]

In [90]:
# k = 0
# for m1,m2 in itertools.product(mm,mm):
#     k += 1
#     print(m1,m2)

In [91]:
def decimalToBinary(n):
    # Converts a given number in decimal representation to a binary number
    return "{0:b}".format(int(n))

In [92]:
def ContainerArray(n,ArraySize):
    # Given a decimal number, this function outputs a binary number
    # in an array of fixed legth.
    # For example, if n=4 and ArraySize=6,
    # The output will be [0,0,0,1,0,0]
    Container = [0 for i in range(ArraySize)]
    BinaryNumber = decimalToBinary(n)
    LenBinary = len(BinaryNumber)
    for i in range(LenBinary):
        Container[ArraySize-1-i] = int(BinaryNumber[LenBinary-1-i])
    return Container

In [93]:
def FockBasisToDecimal(state):
    BinaryString = ''.join(str(e) for e in state)
    n = int(BinaryString,2)
    return n

In [94]:
def BasisGenerator(Nlevels):
    # Given a Hilbert space size, generates all the basis vectors that spans the space
    MaxN = 2**Nlevels # Highest number in the basis in decimal (By highest we mean the fully filled system)
    basis = []
    for i in range(MaxN):
        basis.append(ContainerArray(i,Nlevels))
    return basis

In [95]:
def Bset(Q,cutoff,m):
    # Set of quantum numbers if magentic field and cutoff is specified for a given m
    levels = []
    if ((Q*2) % 2) < 1e-8:
#         M = max(int(abs(m)+0.5),)
        for i in range(-cutoff,cutoff+1):
            l = Q + np.abs(i) - 1/2
            if ((np.abs(m)*2) % 2) > 0.999:
                if np.abs(m) <= np.abs(l):
                    levels.append(i)
    else:
        for i in range(-cutoff,cutoff+1):
            l = Q + np.abs(i) - 1/2
            if ((np.abs(m)*2) % 2) < 1e-8:
                if np.abs(m) <= np.abs(l):
                    levels.append(i)
    return levels

In [96]:
def BasisGeneratorNumberConserving(BasisElement):
    # Given a basis element, this function generates all the other basis elements 
    # of same angular momentum and same particle number
    return list(set(list(itertools.permutations(BasisElement))))

In [97]:
def FockBasisToOperator(BasisElement):
    # Inputs a state in the Fock basis e.g. [0,1,0,1]
    # Outputs it as [2,4]
    basis = []
    for i,element in enumerate(BasisElement):
        if element == 1:
            basis.append(i)
    return basis

In [98]:
def OperatorToFockBasis(state):
    ArraySize = TotalSPStates
    Container = [0 for i in range(ArraySize)]
    for element in state:
        Container[element] = 1
    return Container

In [99]:
def DecimalToFockBasis(n):
    Binary = "{0:b}".format(int(n))
    ArraySize = TotalSPStates
    Container = [0 for i in range(ArraySize)]
    LenBinary = len(Binary)
    for i in range(LenBinary):
        Container[ArraySize-1-i] = int(Binary[LenBinary-1-i])
    return Container

In [100]:
MM = [] # Angular momentum of states
AllStates = [] # Contains tuples of all levels
for m in mm:
    LevelSet = Bset(Q,cutoff,m)
    NN = len(LevelSet)
    for i in range(NN):
        MM.append(m)
    for element in LevelSet:
        AllStates.append((element,m))

In [101]:
# All the indices for two-particle interactions
IntIndices = []
for state1 in AllStates:
    for state2 in AllStates:
        for state3 in AllStates:
            m1 = state1[1]
            m2 = state2[1]
            m3 = state3[1]
            m4 = m1 + m2 - m3
            LevelSet = Bset(Q,cutoff,m4)
            for level in LevelSet:
                IntIndices.append((state1,state2,state3,(m4,level)))

In [102]:
# All the indices for three-particle interactions at cutoff = 0
ThreIn = []
for m1, m2 in itertools.product(mm,mm):
    for m3,m4 in itertools.product(mm,mm):
        for m5 in mm:
            m6 = m1 + m2 + m3 - m4 - m5
            ThreIn.append((m1,m2,m3,m4,m5,m6))
ThreIn = list(set(ThreIn))

In [110]:
ThreIn[6]

(1.5, 2.5, -2.5, -8.5, 3.5, 6.5)

In [105]:
len(ThreIn)

11881376

In [24]:
M1, M2, M3, M4, M5, M6, TInt = np.loadtxt("ThreeParticleIntegralsQ3cutoff0.dat",usecols=(0,1,2,3,4,5,6),unpack=True,delimiter=",")

In [28]:
len(TInt)

4332

In [15]:
# Finds and saves states with the same total angular momentum
data = {}
AMlist = []
ArraySize = len(MM)
for n in range(0,LargestNumber+1):
    Number = ContainerArray(n,ArraySize)
    AngularMomentum = np.dot(MM,Number)
    AMlist.append(AngularMomentum)
    key = str(AngularMomentum)
    if key in data:
        data[key].append(n)
    else:
        data[key] = []
AMs = list(set(AMlist)) # List of all the total angular momentum

del AMlist
gc.collect()

AMfolder = "./DecimalAMCorrCutoff"+str(cutoff)+"Q"+str(Q)
try:
    shutil.rmtree(AMfolder)
except OSError:
    pass

os.mkdir(AMfolder)
    
for key, values in data.items():
    np.savetxt(AMfolder+"/AM"+str(round(float(key),1))+".dat", np.c_[values], fmt="%i") #, np.array(values).astype(int))

del data
gc.collect()

228

In [16]:
TotalL = 0.0
nList = np.loadtxt(AMfolder+"/AM"+str(round(TotalL,1))+".dat", dtype=float).astype(int) # Array of all the decimal numbers corresponding to the angular momentum

In [18]:
nList

array([12, 18, 30, 33, 45, 51, 63])

In [20]:
DecimalToFockBasis(6)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]

In [13]:
OperatorToFockBasis([2,4])

[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
np.binom

In [45]:
FockBasisToOperator([0,0,0,1,0])

[3]

In [13]:
import scipy.special as sp

In [16]:
sp.binom(24,6)

134596.0

In [19]:
24**5 / 1000

7962.624

In [41]:
n = len(Bset(2,2,0.5))

In [42]:
BasisGenerator(n)

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 1, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1],
 [0, 0, 1, 1, 0],
 [0, 0, 1, 1, 1],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 1, 0],
 [0, 1, 0, 1, 1],
 [0, 1, 1, 0, 0],
 [0, 1, 1, 0, 1],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 1],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 1],
 [1, 0, 0, 1, 0],
 [1, 0, 0, 1, 1],
 [1, 0, 1, 0, 0],
 [1, 0, 1, 0, 1],
 [1, 0, 1, 1, 0],
 [1, 0, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 1],
 [1, 1, 0, 1, 0],
 [1, 1, 0, 1, 1],
 [1, 1, 1, 0, 0],
 [1, 1, 1, 0, 1],
 [1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1]]

In [24]:
from sympy import symbols
from sympy.physics.wigner import wigner_6j

In [25]:
wigner_6j(5,5,5,5,5,5)

1/52